In [ ]:
# Installing technical analysis library
# https://technical-analysis-library-in-python.readthedocs.io/en/latest/index.html
!pip install ta

In [ ]:
# Install backtesting infrastructure
# https://pypi.org/project/Backtesting/
!pip install backtesting
!apt-get install nodejs -y
!npm install -g --unsafe-perm ijavascript
!ijsinstall --install=global

In [ ]:
# Preprocess Open, High, Low, Close (OHLC) data for Nifty index from 2009 to 2020
import pandas as pd
nifty_2019 = pd.read_csv('/kaggle/input/indian-stock-index-1minute-data-2008-2020/NIFTY_data/NIFTY_2019.csv', parse_dates=[['Date', 'Time']])
nifty_2019.drop(columns='Instrument', inplace=True)
nifty_2019.set_index('Date_Time',drop=True, inplace=True)
nifty_2019.columns

In [ ]:
# Backtest a simple EMA -> SMA crossover strategy on above data

import backtesting
backtesting.set_bokeh_output(notebook=True)
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

import pandas as pd
import ta.trend as tr


class EmaSmaCross(Strategy):
    def init(self):
        cp = pd.Series(self.data.Close)
        
        self.ema1 = self.I(tr.ema_indicator,close=cp, window=200);
        self.ema2 = self.I(tr.sma_indicator,close=cp, window=2000)
        

    def next(self):
        # Buy or sell on ema -> sma cross over
        # Have a 200 pts stop loss for long trades and 130 points for short trades
        if crossover(self.ema1, self.ema2):
            self.buy(sl=self.data.High-200)
        elif crossover(self.ema2, self.ema1):
            self.sell(sl=self.data.Low+130)
            
bt = Backtest(nifty_2019, EmaSmaCross, commission=.002,cash=1000000,
              exclusive_orders=True)
# Run the backtest
stats = bt.run()
 
# Print the performance metrics for this strategy 
print(stats)

# Plot the price movements, trades and other meta-data such as drawdown etc.
bt.plot(resample=True);

In [ ]:
# List of trades generated by the strategy
stats._trades